In [ ]:
!pip install selenium webdriver-manager pandas

In [ ]:
import time
import pandas as pd
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

opts = Options()
opts.add_argument("--window-size=1600,1000")
opts.add_argument("--disable-gpu")
opts.add_argument("--no-sandbox")
opts.add_argument("--disable-dev-shm-usage")

driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=opts
)

url = "https://yandex.ru/maps/org/peredovyye_platezhnyye_resheniya/103262022758/reviews"
driver.get(url)
time.sleep(10)  # ждём подгрузки JS

prev_height = driver.execute_script("return document.body.scrollHeight")
while True:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(10)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == prev_height:
        break
    prev_height = new_height

info_blocks = driver.find_elements(By.CSS_SELECTOR, "div.business-review-view__info")
body_blocks = driver.find_elements(By.CSS_SELECTOR, "div.business-review-view__body")

print(f"Найдено блоков info: {len(info_blocks)}, блоков body: {len(body_blocks)}")

reviews = []
for info, body in zip(info_blocks, body_blocks):
#     header = driver.find_element(
#     By.CSS_SELECTOR,
#     "div.business-review-view__header"
#     )
#     print(header.get_attribute("outerHTML"))
    # Автор — берем только первую строку
    author_raw = info.find_element(
        By.CSS_SELECTOR,
        "div.business-review-view__author-container"
    ).text
    author = author_raw.splitlines()[0].strip()

    # Дата
    date = info.find_element(
        By.CSS_SELECTOR,
        "div.business-review-view__header"
    ).text.strip()


    # Текст отзыва
    try:
        text = body.find_element(
            By.CSS_SELECTOR,
            "span.business-review-view__body-text"
        ).text.strip()
    except:
        text = ""
    try:
        rating_meta = info.find_element(
             By.CSS_SELECTOR,
             "meta[itemprop='ratingValue']"
         )
        rating = int(float(rating_meta.get_attribute("content")))
    except:
        rating = -1

    reviews.append({
        "author": author,
        "date": date,
        "text": text,
        "rating": rating,
        "source": "yandex"
    })

driver.quit()

df = pd.DataFrame(reviews)
print(df.shape)
df.to_csv("yandex_reviews_local.csv", index=False, encoding="utf-8-sig")
df.tail()


Найдено блоков info: 242, блоков body: 242
(242, 5)


,author,date,text,rating,source
237,Андрей К.,10 марта 2021,Грубияны и зануды.,1,yandex
238,Александр С.,20 августа 2018,Ничего не понравилось. Наоборот закрыл договор...,1,yandex
239,П,24 марта 2016,"Здравствуйте, Аноним!\n\nБлагодарим Вас за отз...",-1,yandex
240,А,4 февраля 2013,Никогда не являлся клиентом компании как физ. ...,1,yandex
241,beerdrinker,17 мая 2018,Один раз настроил всё и работай!,5,yandex


In [ ]:
df.head()

,author,date,text,rating,source
0,Светлана,12 мая,Сотрудничаем с ООО «ППР» не так давно. Хотелос...,5.0,yandex
1,Светлана,11 июня,"Отличная компания, наша организация занимается...",5.0,yandex
2,Tatiyna K.,12 декабря 2024,Сотрудничаем с ООО ППР уже второй год. Хотелос...,5.0,yandex
3,Юлия Логинова,13 февраля 2024,"Отличная компания с удобным сервисом, множеств...",5.0,yandex
4,Юрий Костюк,15 ноября 2024,"Скажу так, работаю с данной компанией больше 3...",5.0,yandex


In [ ]:
import requests
import pandas as pd

def fetch_2gis_branch_reviews(branch_id: str, api_key: str,
                              locale: str = "ru_RU", page_size: int = 50):
    """
    Собирает все отзывы для филиала (branch) 2GIS:
    GET https://public-api.reviews.2gis.com/2.0/branches/{branch_id}/reviews
    :contentReference[oaicite:0]{index=0}
    """
    all_reviews = []
    offset = 0

    while True:
        url = f"https://public-api.reviews.2gis.com/2.0/branches/{branch_id}/reviews"
        params = {
            "key": api_key,
            "locale": locale,
            "limit": page_size,
            "offset": offset,
            # опционально можно добавить:
            # "rated": "true",
            # "sort_by": "date_edited",
            # "without_my_first_review": "false",
        }
        headers = {
            "Accept": "application/json, text/plain, */*",
            "User-Agent": "Mozilla/5.0"
        }
        resp = requests.get(url, params=params, headers=headers, timeout=10)
        resp.raise_for_status()
        data = resp.json()

        # JSON содержит список в ключе "reviews"
        reviews = data.get("reviews", [])
        if not reviews:
            break

        for it in reviews:
            author = (it.get("user") or {}).get("name", "").strip()
            date = it.get("date_created", "")
            # оставляем только дату
            date = date.split("T")[0] if "T" in date else date
            text = (it.get("text") or "").strip()
            rating = it.get("rating")

            all_reviews.append({
                "author": author,
                "date": date,
                "text": text,
                "rating": rating,
                "source": "2gis"
            })

        print(f"Загружено отзывов с offset={offset}: {len(reviews)}")
        offset += page_size

    return pd.DataFrame(all_reviews)

# Пример использования
BRANCH_ID = "4504127916787967"   # ваш ID из URL 2gis
API_KEY   = "***ВАШ API-КЛЮЧ***"

df = fetch_2gis_branch_reviews(BRANCH_ID, API_KEY)
print(df.shape)
df.to_csv("2gis_reviews.csv", index=False, encoding="utf-8-sig")
df.head()


Загружено отзывов с offset=0: 50
Загружено отзывов с offset=50: 39
(89, 5)


,author,date,text,rating,source
0,Евгения Кохтанова,2025-06-09,"Мы юридическое лицо, работаем с ППР уже много ...",5,2gis
1,Алексей Шубин,2025-06-07,"Спасибо Вам, реально облегчели работу с автопа...",5,2gis
2,Ирина Ковалёва,2025-06-06,"Приятно сотрудничать , все вопросы решают быст...",5,2gis
3,Слава Загуменнов,2025-06-06,"Много лет работаю, сервисов все больше, страхо...",5,2gis
4,Ксения Валавина,2025-06-03,"Всегда очень внимательные сотрудники, стараютс...",5,2gis


In [ ]:
df.tail()

,author,date,text,rating,source
84,vkojara,,"Информативный личный кабинет, удобные инструме...",4,2gis
85,PPR,,"Иван, добрый день!\n\nСпасибо за Ваш отзыв. \n...",5,2gis
86,ᴡᴡᴡ.ᴡᴡᴡ-Иван-cloobrbn-ru.okkc.xyz,,"Отвратительная компания, не возможно следить з...",1,2gis
87,staromodov84,,Хочется поделиться своей удивительной историей...,5,2gis
88,kviki173,,"Стали сотрудничать с компанией Петрол Плюс, ка...",4,2gis


In [ ]:
import requests, json

# один запрос на первую «страницу»
url = "https://public-api.reviews.2gis.com/2.0/branches/4504127916787967/reviews"
params = {
    "key": "***ВАШ API-КЛЮЧ***",
    "locale": "ru_RU",
    "limit": 1,
    "offset": 0
}
resp = requests.get(url, params=params)
resp.raise_for_status()
item = resp.json().get("reviews", [])[0]
print(json.dumps(item, ensure_ascii=False, indent=2))


{
  "additional_data": {
    "external_id": null,
    "filtered_facts": null,
    "has_unresolved_complaints": false,
    "ip": "",
    "is_liked": null,
    "is_unrated_permanently": null,
    "pin_status": "none",
    "status_history": [],
    "user_agent": ""
  },
  "comments_count": 1,
  "date_created": "2025-06-09T19:13:37.282873+07:00",
  "date_edited": null,
  "hiding_reason": null,
  "hiding_type": null,
  "id": "163934806",
  "is_hidden": false,
  "is_rated": true,
  "is_verified": false,
  "likes_count": 0,
  "object": {
    "id": "4504127916787967",
    "type": "branch"
  },
  "official_answer": {
    "date_created": "2025-06-09T20:21:35.713235+07:00",
    "id": "48148713",
    "logo_preview_urls": {},
    "org_name": "Передовые платежные решения, процессинговая компания комплексных решений для автопарка",
    "text": "Евгения, добрый день! Благодарим за доверие и выбор нашей компании. Мы приложим все усилия, чтобы и в дальнейшем наше сотрудничество было максимально эффектив

In [ ]:
!pip install requests-html pandas

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.33.0 requires urllib3[socks]~=2.4.0, but you have urllib3 1.26.20 which is incompatible.




   --- ------------------------------------  1/13 [zipp]
   ------ ---------------------------------  2/13 [websockets]
   ------ ---------------------------------  2/13 [websockets]
   ------ ---------------------------------  2/13 [websockets]
   ------ ---------------------------------  2/13 [websockets]
   --------- ------------------------------  3/13 [w3lib]
  Attempting uninstall: urllib3
   --------- ------------------------------  3/13 [w3lib]
    Found existing installation: urllib3 2.4.0
   --------- ------------------------------  3/13 [w3lib]
    Uninstalling urllib3-2.4.0:
   --------- ------------------------------  3/13 [w3lib]
      Successfully uninstalled urllib3-2.4.0
   --------- ------------------------------  3/13 [w3lib]
   ------------ ---------------------------  4/13 [urllib3]
   ------------ ---------------------------  4/13 [urllib3]
   ------------ ---------------------------  4/13 [urllib3]
   ------------ ---------------------------  4/13 [urllib3]
   

In [ ]:
import pandas as pd

# # 1. Загружаем уже полученные CSV (Yandex и 2GIS)
# df_yandex = pd.read_csv("yandex_reviews_local.csv", encoding="utf-8-sig")
# df_2gis   = pd.read_csv("2gis_reviews.csv",      encoding="utf-8-sig")

google_reviews = [
    {
        "author": "Василь Куруц",
        "date":   "2025",
        "text":   "Хочу выразить благодарность компании ППР и лично менеджеру Наталье Савичевой за отличную работу и удобство предоставляемых услуг. Мы ценим вашу работу и высокий профессионализм . Отдельно хотим отметить удобство использования каршеринга  - это очень удобно и выгодно для нас!",
        "rating": 5,
        "source": "google"
    },
    {
        "author": "Максим Адеев",
        "date":   "2019",
        "text":   "Благодарен компании за очень удобные сервисы, личный кабинет, возможность выгрузки данных в наши системы и менеджеру за поддержку в любое время суток. Хорошо, что вы все время предлагаете что то новое для работы с топливными картами. Клиент ориентированность на высшем уровне!",
        "rating": 5,
        "source": "google"
    },
    {
        "author": "Zhanna Davoyan",
        "date":   "2022",
        "text":   "пользуемся в компании картами ППР года три. удобно то, что можно оплачивать все, что связано с автопарком. водителям удобно заправляться, потому почти везде принимают карты и еще шиномонтаж и мойки тоже по карте можно оплатить.",
        "rating": 5,
        "source": "google"
    },
    {
        "author": "Jean Anatole",
        "date":   "2019",
        "text":   "Однажды внезапно заблокировали все аккаунты нашей компании для входа в ЛК. Техподдержка сообщила, что заблокировал кто-то из нас, и предложил быстро разблокировать аккаунты, попросил прислать им заявление по электронной почте. Заявление отправил, через два дня нас так и не разблокировали, снова звоню в техподдержку, говорят, что заявления нет, разговор длился 20 минут, в итоге заявление все таки нашли и доступ разблокировали. Написал заявление с просьбой выяснить причину по которой вообще произошла блокировка, через день перезвонили и сказали что блокировок не было, и еще через день перезвонили и сказали, что к сожалению логи хранятся 4 дня и причины блокировки выяснить уже не удаётся. Вот так вот ) UPD: Отправил данные как попросили в комментарии, перезвонили через 5 дней, сказали что ситуация не изменилась и логи хранятся только 4 дня. Просто они ко всем отзывам оставляют свой стандартный коммент.",
        "rating": 1,
        "source": "google"
    },
    {
        "author": "Евгений Кононенков",
        "date":   "2019",
        "text":   "Лидер на рынке топливных карт! Самый лучший сервис и самые инновационные подходы к бизнесу!",
        "rating": 5,
        "source": "google"
    },
    {
        "author": "Павел Андреев",
        "date":   "2022",
        "text":   "Есть все, что нужно, но дружелюбность интерфейса оставляет желать лучшего.",
        "rating": 4,
        "source": "google"
    },
    {
        "author": "Роман М",
        "date":   "2022",
        "text":   "Отличный офис",
        "rating": 5,
        "source": "google"
    },
    {
        "author": "Artem Parasochka",
        "date":   "2018",
        "text":   "Всё круто!",
        "rating": 5,
        "source": "google"
    }
]

df_google = pd.DataFrame(google_reviews)

# 4. Сохраняем итоговый CSV
df_google.to_csv("google_reviews.csv", index=False, encoding="utf-8-sig")
print("Всего строк в объединённом CSV:", len(df_google))


Всего строк в объединённом CSV: 8
